In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
class VGG19(nn.Module):
    def __init__(self, num_classes):
        super(VGG19, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), 
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [5]:
import torchvision

In [6]:
from torchvision import datasets, transforms

In [7]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
def image_processing(image_path,size=(224,224)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(size)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]) 
    tensor = transform(image)
    tensor = tensor.unsqueeze(0)
    tensor = tensor.float()
    return tensor

In [15]:
class ImageDataset(Dataset):
    def __init__(self,csv_file):
        self.data = pd.read_csv(csv_file).iloc[:5000]
        self.additional_data = []
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img_path = f"Downloads/cifar100-image-classification/{self.data.iloc[idx,0]}"
        label = self.data.iloc[idx, 1]
        image = image_processing(img_path)
        return image,label
    def append_data(self, image_tensor, label):
        self.additional_data.append((image_tensor, label))

In [17]:
csv_file = "Downloads/cifar100dataset.csv"
dataset = ImageDataset(csv_file)
dataloader = DataLoader(dataset,batch_size=25,shuffle=True)

In [19]:
num_classes = len(dataset.data['cattle'].unique()) 

In [21]:
num_classes

100

In [23]:
Custommodel = VGG19(num_classes) 

In [25]:
classes = dataset.data['cattle'].unique()

In [27]:
class_to_idx = {classes[i]: i for i in range(len(classes))}

In [29]:
idx_to_class = {i:classes[i] for i in range(len(classes))}

In [ ]:
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(Custommodel.parameters(),lr=0.005)

for epoch in range(20):
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs,label in dataset:
        optimizer.zero_grad()
        outputs = Custommodel(inputs)
        labelidx = class_to_idx[label]
        labelidx = torch.tensor([labelidx])
        labelidx = labelidx.long()
        loss = criterion(outputs,labelidx)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        k,predicted = torch.max(outputs, 1)
        total += 1
        correct += (predicted.item() == labelidx.item())
    print(f"Epoch {epoch + 1}/{20}, Loss: {running_loss/len(dataset)}, Accuracy: {100 * correct / total}%")

In [ ]:
class Vgg19(nn.Module):
    def __init__(self,model):
        super(VGG19ForStyleTransfer,self).__init__()
        self.model = model
        self.required_layers = [0,5,10,19,28] 
    def extract_features(self,x):
        features = []
        for layer_num,layer in enumerate(self.model.features):
            x = layer(x)
            if layer_num in self.required_layers:
                features.append(x)
        return features
    def forward(self, x):
        features = self.extract_features(x)
        return features

In [ ]:
styletransfer = Vgg19(Custommodel)

In [ ]:
def content_loss(content_feature,noise_feature):
    loss = 0
    for content_f,noise_f in zip(content_features,noise_features):
        loss+=(torch.mean((content_f-noise_f)**2))
    return loss

In [ ]:
def gram_matrix(features):
    b,c,h,w = features.size()
    features = features.view(c,h*w)
    grammatrix = torch.mm(features,features.t())
    return grammatrix

In [ ]:
def style_loss(style_feature,noise_feature):
    loss = 0
    for style_f,noise_f in zip(style_feature,noise_feature):
        gram_style = gram_matrix(style_f)
        gram_noise = gram_matrix(noise_f)
        N = gram_style.size(0)
        M = gram_noise.size(1)
        loss+=(torch.mean((gram_style-gram_noise)**2))
    return loss

In [ ]:
content_w = 8
style_w = 40

In [ ]:
def total_loss(content_w,style_w,content_l,style_l):
    loss = (content_w*content_l)+(style_w*style_l)
    return loss

In [ ]:
content_image_path = 'Downloads/Tesla_circa_1890.jpeg'
style_path = 'Downloads/style_image.jpg'

In [ ]:
style_image = image_processing(style_path)

In [ ]:
style_image.shape

In [ ]:
output_path = 'Downloads/output_style_images'
os.makedirs(output_path,exist_ok=True)

In [ ]:
content_image = image_processing(content_image_path)
noise_image = content_image.clone().requires_grad_(True)

In [ ]:
optimizer2=optim.Adam([noise_image],lr=0.01)

In [ ]:
def deprocess_image(image_tensor):
    image_tensor = image_tensor.clone().squeeze(0)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    image_tensor = image_tensor * std + mean
    image_tensor = image_tensor.clamp(0, 1)
    image_np = image_tensor.permute(1, 2, 0).cpu().numpy()
    return cv2.cvtColor((image_np * 255).astype('uint8'), cv2.COLOR_RGB2BGR)

In [ ]:
for i in range(3000):
    optimizer2.zero_grad()
    content_features = styletransfer(content_image)
    style_features = styletransfer(style_image)
    noise_features = styletransfer(noise_image)
    content_l = content_loss(content_features,noise_features)
    style_l = style_loss(style_features, noise_features)
    t_loss = total_loss(content_w, style_w, content_l, style_l)
    t_loss.backward()
    optimizer2.step()
output_image = deprocess_image(noise_image)
output_image_name = "Output_stylized_img.jpg"
cv2.imwrite(os.path.join(output_path,output_image_name),output_image)
print("Style transfer complete")